In [1]:
import json
import pandas as pd

# Load player season history and create features

In [2]:
# Load all player season history into a single data frame
with open('data/during-season/bootstrap-static.json', 'r') as json_file:
    data = json.loads(json_file.read())
    
df_player_overview = pd.json_normalize(data, record_path='elements')

df_players_history = pd.DataFrame()
for index, player in df_player_overview.iterrows():
    player_id = str(player['id'])
    first_name = player['first_name']
    last_name = player['second_name']    
    file_name = f"{player_id.rjust(3, '0')}_{first_name}_{last_name}".replace(' ', '_')

    with open(f'data/during-season/players/{file_name}.json', 'r') as json_file:
        data = json.loads(json_file.read())
        df_player_history = pd.json_normalize(data, record_path='history')
        df_players_history = df_players_history.append(df_player_history)

df_players_history.shape

(862, 33)

In [3]:
df_players_history.to_csv('data/during-season/player_history.csv', index=False)